In [ ]:
from airfoil.util.shapely_helpers import plot_shapely_directional
from airfoil.util.linestring_helpers import (
    deflection_angle_padded,
    resample_long_segments,
)
from airfoil.util.array_helpers import map_to_range
from functools import reduce
from shapely import box, difference, union, Point
from shapely.affinity import translate, scale
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
bw=24
bh=43

ox = -10
oc = 20

w = 65.5
h = 28.3

f = difference(
    box(
        -w/2,-h,
         w/2, 0,
    ),
    reduce(union,[
        box(
            -bw/2+ox,-bh/2,
             bw/2+ox, bh/2,
        ),
        Point(-bw/2+ox,-bh/2).buffer(0.8, quad_segs=3),
        Point( bw/2+ox,-bh/2).buffer(0.8, quad_segs=3),
        Point(oc,0).buffer(16/2, quad_segs=8)
    ])
)
f = scale(f,-1,1)
f=translate(f, w/2, 0)
plot_shapely_directional([f])

In [ ]:
c = np.array(f.exterior.coords)[4:-1]
c[c[:,1]==c[:,1].max(),1]+=2
c = resample_long_segments(c,1)
fig,ax = plt.subplots()

ax.plot(*c.T,"-o")
ax.set_aspect("equal")

In [ ]:
speed = map_to_range(deflection_angle_padded(c),250,100)

lead_in = np.array([
    [-20,  0],
    [-20, 50],
    [  0, 50],
    [c[0,0], c[0,1]+1],
])

lead_out = np.array([
    [c[-1,0], c[-1,1]+1],
    [c[-1,0], c[-1,1] + 20]
])
lead_in  = resample_long_segments(lead_in,1)
lead_out = resample_long_segments(lead_out,1)

commands = np.concat([
    np.insert(np.tile(lead_in , 2), 0,   500, axis=-1),
    np.insert(np.tile(c       , 2), 0, speed, axis=-1),
    np.insert(np.tile(lead_out, 2), 0,   500, axis=-1),
])

In [ ]:
from matplotlib.collections import LineCollection
def plot_feedrate_path(data, title="Feed Rate Path", figsize=(10, 8)):
    """
    Plot a path with line segments colored by feed rate.
    
    Parameters:
    -----------
    data : numpy.ndarray
        Array of shape (n, 3) where each row is (s, x, y)
        s = feed rate for the next line segment
        x, y = coordinates
    title : str, optional
        Title for the plot
    figsize : tuple, optional
        Figure size (width, height)
    
    Returns:
    --------
    fig, ax : matplotlib figure and axes objects
    """
    
    if data.shape[1] != 3:
        raise ValueError("Data must have shape (n, 3) with columns (s, x, y)")
    
    # Extract components
    feed_rates = data[:, 0]
    x_coords = data[:, 1]
    y_coords = data[:, 2]
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create line segments for coloring
    # Each segment connects consecutive points
    points = np.column_stack([x_coords, y_coords])
    segments = np.array([points[:-1], points[1:]]).transpose(1, 0, 2)
    
    # Use feed rates for coloring (excluding the last point since it doesn't have a "next" segment)
    colors = feed_rates[:-1]
    
    # Create LineCollection with viridis colormap
    lc = LineCollection(segments, cmap='viridis', linewidths=2)
    lc.set_array(colors)
    
    # Add the line collection to the plot
    line = ax.add_collection(lc)
    
    # Add scatter plot for points
    scatter = ax.scatter(x_coords, y_coords, c=feed_rates, cmap='viridis', 
                        s=30, edgecolors='black', linewidth=0.5, zorder=5)
    
    # Add colorbar
    cbar = plt.colorbar(line, ax=ax)
    cbar.set_label('Feed Rate (s)', rotation=270, labelpad=20)
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title(title)
    
    # Set equal aspect ratio for better visualization
    ax.set_aspect('equal', adjustable='box')
    
    # Add grid
    ax.grid(True, alpha=0.3)
    
    # Adjust layout
    plt.tight_layout()
    
    return fig, ax

In [ ]:
plot_feedrate_path(commands[:,:3])

In [ ]:
from airfoil.cnc import CNC
cnc=CNC()

In [ ]:
cnc.metric()
cnc.absolute()
cnc.home()
cnc.set_position(0,0,0,0)

In [ ]:
cnc.travel(
    x=110,y=25,
    z=100,a=25
)

In [ ]:
cnc.set_position(0,0,0,0)
cnc.absolute()

In [ ]:
cnc.travel(0,12,0,12)

In [ ]:
cnc.feed(250,-80,0,-80,0)

In [ ]:
cnc.send_g1_commands(commands)